In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.stats

import sys
sys.path.append("../../../")

In [3]:
from chiseling.dgps.basic_binary_regression import BasicBinaryRegression
from chiseling.source.learners.baselearners_binary import logreg_learner
from chiseling.source.strategies.simul_data_splitting_strategy import SimulDataSplittingStrategy

In [4]:
# Hyperparameters
n = 1000
d = 100
s = 5
rho = 0.2
theta = 2
tau = 2.2

test_thresh = 0.9

alpha = 0.05

random_seed = 1000

In [5]:
# Sample data
sampler = BasicBinaryRegression(d, s, rho, theta, tau, random_seed=random_seed)

### Check coverage

In [6]:
np.random.seed(random_seed)

In [7]:
n_sims = 300

In [8]:
all_rejections = []
for _ in range(n_sims):
    if _ % 50 == 0:
        print("Progress", _)
    # Sample data
    X, Y = sampler.sample(n)
    # Run data splitting
    rs = np.random.randint(0, 2**32-1)
    simul_datasplit = SimulDataSplittingStrategy(X=X,
                                                 Y=Y,
                                                 train_ratio=0.5,
                                                 learner=logreg_learner,
                                                 alpha=alpha,
                                                 test_thresh=test_thresh,
                                                 n_min=30,
                                                 binary=True,
                                                 randomize=True,
                                                 random_seed=rs)
    simul_datasplit.run_strategy()
    # Data split rejection
    all_rejections.append(simul_datasplit.rejected)
all_rejections = np.array(all_rejections)

Progress 0
Progress 50
Progress 100
Progress 150
Progress 200
Progress 250


In [9]:
all_rejections.mean(), all_rejections.std() / np.sqrt(n_sims)

(np.float64(0.9), np.float64(0.01732050807568877))

In [10]:
all_rejections.mean() - 2 * (all_rejections.std() / np.sqrt(n_sims))

np.float64(0.8653589838486225)

In [11]:
all_rejections.mean() + 2 * (all_rejections.std() / np.sqrt(n_sims))

np.float64(0.9346410161513775)

### Benchmark module

In [12]:
# from chiseling.benchmark.benchmark import Benchmark

In [13]:
# settings = {'dgp': 'BasicBinaryRegression',
#  'n': 1000,
#  'd': 100,
#  's': 5,
#  'rho': 0.2,
#  'theta': 2,
#  'tau': 2.2,
#  'alpha': 0.05,
#  'test_thresh': 0.9,
#  'learner': 'logreg_learner',
#  'binary': True,
#  'strategy': 'SimulDataSplittingStrategy',
#  'n_min': 20,
#  'train_ratio': 0.5,
#  'n_sims': 100,
#  'task_id': 900,
#  'random_seed': 420}

In [14]:
# benchmark = Benchmark(settings)

In [15]:
# benchmark.simulate_batch()

In [16]:
# benchmark.simulation_results_df.rejected.mean()

In [17]:
# benchmark.simulation_results_df.rejected.std() / np.sqrt(benchmark.simulation_results_df.shape[0])